# In The Mood For Love

Projeto inspirado pelo filme In The Mood For Love do diretor Wong Kar-Wai.

Passos:
1. Extrair trecho do filme e separar em cenas. Duas opções: utilizar o método do ffmpeg ou a biblioteca scenedetect.
2. Extrair frames das cenas para analisar imagem (hue, saturação...).
3. Extrair áudio das cenas e coletar informações como nível médio de áudio e silêncio.

Fazer funcionar antes para depois melhorar.
Pontos a melhorar: adicionar mais atributos de audio e de imagem, extrair mais de um frame por cena, otimizar detecção de cenas...

In [ ]:
import pandas as pd

In [ ]:
from scene_detection import detect_scene
from feature_extraction import audio_features

In [ ]:
video_path = '../data/' 
video_name = 'inthemood.mp4' 

df = pd.DataFrame() 
df = detect_scene(video_name) 

In [ ]:
df = audio_features('...', video_name, df) 

In [ ]:
# df = extract_frames() 
# df = image_features() 